In [ ]:
#| hide
import os
import joblib

# Syncing through nbdev

> Showcasing the syncing capability

In [ ]:
%load_ext nbmodular.core.cell2func

In [ ]:
%%function myf
print ('hello')
a = 3

> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(243)run_code_and_collect_locals()
    241         if not is_test_function:
    242             pdb.set_trace()
--> 243             get_old_variables_code = f'\nfrom nbmodular.core.cell2func import keep_variables\nkeep_variables ("previous_values", locals ())'
    244             get_ipython().run_cell(get_old_variables_code)
    245             if store_values:



ipdb>  c


hello
> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(287)match_variables_and_locals()
    285         pdb.set_trace()
    286         # previous variables / values
--> 287         self.previous_variables = [k for k in self.previous_variables if k in self.previous_values]
    288         self.previous_variables += [k for k in self.argument_variables if k in self.previous_values and k not in self.previous_variables]
    289         self.previous_variables += [k for k in self.created_variables if k in self.previous_values and k in self.loaded_names+self.argument_variables and k not in self.previous_variables]



ipdb>  c


Stored the following local variables in the myf current_values dictionary: ['a']
> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(316)match_variables_and_locals()
    314         pdb.set_trace()
    315         keys_update_code=(
--> 316 f'''
    317 import joblib
    318 keys = joblib.load ('function_processor_keys.pk')



ipdb>  c


In [ ]:
%print myf

def myf():
    print ('hello')
    a = 3
    return a



In [ ]:
myf_info.original_cell

"print ('hello')\na = 3\n"

In [ ]:
%%function myf --merge
b = 4

In [ ]:
%% function myf --merge
c = 5
d = a+b+c

{'input': None, 'unknown_input': True, 'output': None, 'unknown_output': True}
